# Training a Convolutional Neural Network (CNN) on CIFAR-10 Using TensorFlow (Transfer Learning)

This project introduces foundational concepts of deep learning with TensorFlow by building, training, and evaluating a Convolutional Neural Network (CNN) on the CIFAR-10 dataset. The goal is to classify 32x32 RGB images into 10 distinct categories (e.g., airplanes, cars, birds). Participants will gain hands-on experience with TensorFlow workflows, CNN architectures, and image classification techniques.

Model Performance: Achieve > 70% test accuracy (baseline).

In [ ]:
# Import the necessary libraries
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras

In [ ]:
# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Load the dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [ ]:
class_names = ['airplane','automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Split validation set from train images
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.10, random_state=42)

**Preprocessing**

Using ResNet50 model. For preprocessing, it is better to resize the image to the shape of the image, the base model was trained in. that's why I resized the 32x32 to 224x224. Also normalize it to the way resnet50 normalizes the images [1, -1]. In the first notebook, i normalized it to [0, 1] but when using a base model, i have learned that you should preprocess to make the images as close to the orginal images used to train the model, as possible.

In [ ]:
# Preprocess
def preprocess_and_augment(image, target_size=(224, 224)):
  # Resize the image to 224x224 (ResNet50 input size)
  image = tf.image.resize(image, target_size)

  # Normalize the image using ResNet50 preprocessing
  image = keras.applications.resnet50.preprocess_input(image)

  # Apply data augmentation
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  image = tf.image.rot90(image, k=tf.random.uniform([], minval=0, maxval=4, dtype=tf.int32))
  image = tf.image.random_contrast(image, lower=0.2, upper=0.5)

  return image


To apply the preprocessing, I wasn't able to apply it all at once without getting "LimitedResources" error. So, I processed it in batches. To be able to do this, I changed the numpy datase to tensorflow dataset so that they can be processed in batches

In [ ]:
# Create a dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))

# Apply transformations
train_dataset = train_dataset.map(lambda x, y: (preprocess_and_augment(x), y), num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(lambda x, y: (preprocess_and_augment(x), y), num_parallel_calls=tf.data.AUTOTUNE)

# Batch the dataset and optimize performance with prefetch
train_dataset = train_dataset.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)

**Building the Model**

In [ ]:
base_model = keras.applications.ResNet50(include_top=False, weights="imagenet", pooling="avg")

base_model.trainable = False # Freeze the weights of the top layers first

model = keras.models.Sequential([
    keras.layers.InputLayer(shape=[224, 224, 3]),
    base_model,
    keras.layers.Dense(units=128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(units=10, activation="softmax")
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 2048)                │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Train 1
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 141s 93ms/step - accuracy: 0.6223 - loss: 1.2527 - val_accuracy: 0.7058 - val_loss: 0.9450
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 87ms/step - accuracy: 0.7271 - loss: 0.8876 - val_accuracy: 0.7184 - val_loss: 0.9164
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 87ms/step - accuracy: 0.7398 - loss: 0.8504 - val_accuracy: 0.7174 - val_loss: 0.9060
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 87ms/step - accuracy: 0.7488 - loss: 0.8250 - val_accuracy: 0.7244 - val_loss: 0.8914
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 88ms/step - accuracy: 0.7513 - loss: 0.8127 - val_accuracy: 0.7392 - val_loss: 0.8484
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 87ms/step - accuracy: 0.7550 - loss: 0.8070 - val_accuracy: 0.7494 - val_loss: 0.8352
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 87ms/step - accuracy: 0.7574 - loss: 0.8016 - val_accuracy: 0.7370 - val_loss: 0.8634
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 123s 87ms/step - accuracy: 

In [ ]:
base_model.layers[-10].trainable = True # Unfreeze the last 10 layers to finetune

# ReCompile the model
model.compile(optimizer="adam", loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Set up callbacks
my_callbacks = [
    keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint("cifar10_tl.keras", save_best_only=True),
]

# Train 2
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=my_callbacks)

Epoch 1/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 145s 96ms/step - accuracy: 0.7831 - loss: 0.7208 - val_accuracy: 0.7538 - val_loss: 0.8238
Epoch 2/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 126s 90ms/step - accuracy: 0.7798 - loss: 0.7298 - val_accuracy: 0.7486 - val_loss: 0.8414
Epoch 3/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 127s 90ms/step - accuracy: 0.7818 - loss: 0.7390 - val_accuracy: 0.7564 - val_loss: 0.8222
Epoch 4/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 126s 90ms/step - accuracy: 0.7817 - loss: 0.7366 - val_accuracy: 0.7572 - val_loss: 0.8298
Epoch 5/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 126s 90ms/step - accuracy: 0.7841 - loss: 0.7331 - val_accuracy: 0.7464 - val_loss: 0.8498
Epoch 6/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 126s 90ms/step - accuracy: 0.7856 - loss: 0.7300 - val_accuracy: 0.7552 - val_loss: 0.8445
Epoch 7/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 126s 90ms/step - accuracy: 0.7885 - loss: 0.7203 - val_accuracy: 0.7542 - val_loss: 0.8363
Epoch 8/100
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 126s 90ms/step - ac

Train Accuracy: 79%
Val Accuracy: 75%

It doesn't seem to be overfitting. Slight increase from my custom model, although I wish it could be more. Let's evaluate on the test model

**Evaluation**

First, preprocess the model to look like the resnet expected input size and normalize it too.

In [ ]:
# Preprocess
def preprocess(image, target_size=(224, 224)):
  # Resize the image to 224x224 (ResNet50 input size)
  image = tf.image.resize(image, target_size)

  # Normalize the image using ResNet50 preprocessing
  image = keras.applications.resnet50.preprocess_input(image)

  return image

# Create a dataset for the test set
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

test_dataset = test_dataset.map(lambda x, y: (preprocess(x), y), num_parallel_calls=tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Now evaluate on the test model
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 29s 91ms/step - accuracy: 0.8124 - loss: 0.6832
Test Loss: 0.6791001558303833, Test Accuracy: 0.8130000233650208


Test Accuracy: 81%

This is much better than 69% of my custom model. Which is good.